# Измерение качества моделей

## Метрики качества

Это практическое задание посвящено ознакомлению с инструментами оценки качества моделей машинного обучения, которые предоставляет библиотека `scikit-learn`. Метрики качества, представленные различными функциями, находятся в модуле `sklearn.metrics`. Мы начнем с загрузки набора данных из файла `data.csv` при помощи функции `read_csv` из библиотеки `pandas`. Этот набор данных содержит информацию о предсказаниях различных алгоритмов машинного обучения для решения задачи классификации. Колонка `prediction` - это результаты работы одного из этих алгоритмов. Целевая переменная содержится в столбце `target` (класс 0 или 1). Подсчитайте значение `true negative`, `false negative`, `true positive` и `false positive`. Запишите эти значения через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой. Далее, посчитайте для этих данных значение таких метрик как `precision`, `recall` и `f1 score` с точностью до двух знаков после запятой. Запишите результаты в строго заданном порядке через запятую в переменную `answer2`, которая так же будет являться строкой

### *РЕШЕНИЕ*

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('data.csv')
df.head()

,prediction,scores_1,scores_2,target
0,1,0.89,0.9,1
1,0,0.06,0.0,0
2,1,0.71,0.6,0
3,1,1.00,0.7,1
4,1,1.00,1.0,1


In [71]:
tn, fp, fn, tp = confusion_matrix(df.target, df.prediction).ravel()

In [72]:
answer1 = ','.join([str(round(tn, 0)), str(round(fn, 0)), str(round(tp, 0)), str(round(fp, 0))])

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [16]:
pre_s = precision_score(df.target, df.prediction)

In [17]:
rec_s = recall_score(df.target, df.prediction)

In [18]:
f1_s = f1_score(df.target, df.prediction)

In [22]:
answer2 = ','.join([str(round(pre_s, 2)), str(round(rec_s,2)), str(round(f1_s,2))])

В столбцах `scores_1` и `scores_2` содержаться оценки вероятности пренадлежности объектов к классу 1 для двух разных алгоритмов машинного обучения. Рассчитайте площадь под ROC-кривой для каждого алгоритма и сравните их. В качестве ответа `answer3` приведите большее из двух значений, округленное до трех знаков после запятой.

### *РЕШЕНИЕ*

In [24]:
from sklearn.metrics import roc_auc_score

In [25]:
alg1 = roc_auc_score(df.target, df.scores_1)
alg2 = roc_auc_score(df.target, df.scores_2)

In [28]:
answer3 = max(alg1, alg2)

## Метод скользящего контроля

Во второй части данного практического задания мы изучать различные методы оценки моделей машинного обучения. Загрузите набор данных `Breast Cancer Wisconsin (Diagnostic)`, используя функцию `load_breast_cancer` из модуля `sklearn.datasets`. Этот датасет позволяет решать задачу предсказания рака груди по различным характеристикам опухоли. В данном случае, целевая переменная принимает два значения, соответствующие доброкачественной и злокачественной опухоли. Проверьте, является ли данная выборка сбалансированной.

In [29]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)
counts = pd.value_counts(y)
print("Is this a balanced dataset? {}".format(counts[1] == counts[0]))

Is this a balanced dataset? False


In [31]:
counts

1    357
0    212
dtype: int64

Первый метод, который мы будем использовать, - это случайное разбиение датасета на тренировочную и тестовую выборку с помощью функции `train_test_split` из `sklearn.model_selection` с параметрами `random_state=3` и `test_size=0.33`. Если выборка является несбалансированной передайте целевую переменную в эту функцию в качестве аргумента `stratify`.

Обучите логистическую регрессию (класс `LogisticRegression` из модуля `sklearn.linear_model`) с параметром конструктора `random_state=42` и метод K ближайших соседей (класс `KNeighborsClassifier` из модуля `sklearn.neighbors`) на тренировочной выборке. Оцените качество на тестовой выборке для каждой из моделей. В качестве метрики качества используйте `recall`. Какая из моделей показывает лучший результат? Ответом на это задание `answer4` является этот результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionReceiver Operating Characteristic (ROC) with cross validation¶

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=3, stratify=y)

In [36]:
lr_clf = LogisticRegression(random_state=42)
knn_clf = KNeighborsClassifier()

In [37]:
lr_pred = lr_clf.fit(X_train, y_train).predict(X_test)
knn_pred = knn_clf.fit(X_train, y_train).predict(X_test)

/home/zark/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
answer4 = max(recall_score(y_test, lr_pred), recall_score(y_test, knn_pred))

Далее мы проведем оценку каждой из этих моделей в соответствии с методом скользящего контроля с помощью функции `cross_val_score` из модуля `sklearn.model_selection`. В качестве параметра кросс-валидации `cv` в этой функции используйте экземпляр класса `StratifiedKFold` из `sklearn.model_selection` с тремя разбиениями. 

Функция `cross_val_score` возвращает количество оценок, соответствующие числу разбиений.
В качестве итогового результата используете среднее значение полученных оценок с помощью метрики `recall`. Какая модель работает лучше в это случае? Какие выводы можно из этого сделать? Ответом на это задание `answer5` является лучший итоговый результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [44]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [50]:
cv = StratifiedKFold(n_splits=3)

In [59]:
cvs_lr = cross_val_score(lr_clf, X, y, cv=cv, scoring='recall')
cvs_knn = cross_val_score(knn_clf, X, y, cv=cv, scoring='recall')

/home/zark/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/zark/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n

In [62]:
answer5 = max(cvs_lr.mean(), cvs_knn.mean())

# Строка с ответами

In [74]:
output = """TN,FN,TP,FP = {0}
Precision,Recall,F1 Score = {1}
Best ROC AUC Score {2:.3f}
Random Split {3:.3f}
Cross Val Score {4:.3f}"""
print(output.format(answer1, answer2, answer3, answer4, answer5))

TN,FN,TP,FP = 63,4,114,7
Precision,Recall,F1 Score = 0.94,0.97,0.95
Best ROC AUC Score 0.993
Random Split 0.975
Cross Val Score 0.964
